### Connect All API Keys

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [4]:
# Load the GROQ API Key
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

### Video Link Summarization using Map Reduce Method

In [5]:
# Import required libraries
import numpy as np
from langchain.document_loaders import YoutubeLoader
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import load_summarize_chain
from langchain.schema import Document

In [6]:
url = "https://www.youtube.com/watch?v=uTSGHBsvplg&t=123s"

In [7]:
#To check the function for in which language transcript is avilable

def get_transcript_languages(youtube_video_url):
    try:
        video_id = youtube_video_url.split("v=")[-1].split("&")[0]
        languages = YouTubeTranscriptApi.list_transcripts(video_id)
        return [lang.language for lang in languages]
    except TranscriptsDisabled:
        return "Subtitles are disabled for this video."
    except Exception as e:
        return f"Error fetching transcript languages: {e}"

In [8]:
print(get_transcript_languages(url))

['Hindi (auto-generated)']


In [9]:
# Language selection

select_lang = get_transcript_languages(url)[0][:2].lower()
print(select_lang)

hi


In [10]:
loader = YoutubeLoader.from_youtube_url(url,language=[select_lang],translation=select_lang)
# loader = YoutubeLoader.from_youtube_url(url,translation="gu")
docs = loader.load()
docs

[Document(metadata={'source': 'uTSGHBsvplg'}, page_content='[संगीत] पीटर पांडे के पास एक तोता यानी कि पैरोट है जिसका नाम है बड़ी बड़ी की मेमोरी बहुत ही शार्प है और उसको चीजों को कॉपी करना बहुत अच्छा आता है बहुत अच्छा आता है बड़ी पीटर के घर में जो भी कन्वर्जेन चल रहे है उसको सुनता रहता है और उसको कॉपी करने की क्षमता रखता है जब उसको यह सुनाई देता है कि भैया मुझे भूख लगी है मुझे चाहिए कुछ बिरयानी इस केस में बड़ी मोस्ट लाइक बिरयानी या तो फिर चेरीज या तो फिर फूड ऐसा बोलेगा क्योंकि पास में उसने ऐसी चीजों को सुना है वो यह नहीं बोलेगा कि मुझे बाइसिकल चाहिए या फिर बुक चाहिए जब किसी को भूख लगी है तो बड़ी को बिरयानी फूड बाइसिकल इन सब चीजों का मीनिंग नहीं पता है लेकिन वो एक स्टेटस्ट प्रोबेबिलिटी को यूज करता है बेस्ड ऑन जितने कन्वर्सेशन उसने पास्ट में सुने हैं और वह उसके बेसिस पे स्टेटिस्टिक्स दे देता है बड़ी को हम स्टॉकस्टेटमेंट के बेस प चलती है और थोड़ा रेंडम नेस को भी यूज कर लेती है लैंग्वेज मॉडल है ना वह एक स्टॉकस्टेटमेंट प्रोग्राम होता है जो अंडर निथ न्यूरल नेटवर्क करके एक टेक्नोलॉजी यूज करता 

In [11]:
#Text details avilable in video link

transcript = docs[0].page_content
print(transcript)

[संगीत] पीटर पांडे के पास एक तोता यानी कि पैरोट है जिसका नाम है बड़ी बड़ी की मेमोरी बहुत ही शार्प है और उसको चीजों को कॉपी करना बहुत अच्छा आता है बहुत अच्छा आता है बड़ी पीटर के घर में जो भी कन्वर्जेन चल रहे है उसको सुनता रहता है और उसको कॉपी करने की क्षमता रखता है जब उसको यह सुनाई देता है कि भैया मुझे भूख लगी है मुझे चाहिए कुछ बिरयानी इस केस में बड़ी मोस्ट लाइक बिरयानी या तो फिर चेरीज या तो फिर फूड ऐसा बोलेगा क्योंकि पास में उसने ऐसी चीजों को सुना है वो यह नहीं बोलेगा कि मुझे बाइसिकल चाहिए या फिर बुक चाहिए जब किसी को भूख लगी है तो बड़ी को बिरयानी फूड बाइसिकल इन सब चीजों का मीनिंग नहीं पता है लेकिन वो एक स्टेटस्ट प्रोबेबिलिटी को यूज करता है बेस्ड ऑन जितने कन्वर्सेशन उसने पास्ट में सुने हैं और वह उसके बेसिस पे स्टेटिस्टिक्स दे देता है बड़ी को हम स्टॉकस्टेटमेंट के बेस प चलती है और थोड़ा रेंडम नेस को भी यूज कर लेती है लैंग्वेज मॉडल है ना वह एक स्टॉकस्टेटमेंट प्रोग्राम होता है जो अंडर निथ न्यूरल नेटवर्क करके एक टेक्नोलॉजी यूज करता है और उसको आप कोई भी सेंटेंस दो तो उस सेंटेंस को वोह ऑटो कंप

In [12]:
#Show the information regarding video metadata

for key, value in docs[0].metadata.items():
    print(f"{key}: {value}")

source: uTSGHBsvplg


In [13]:
#Total number of characters avilable in text

length = len(transcript)
print(length)

5072


In [14]:
# Split the transcript into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=len(transcript)/5, chunk_overlap=50)
text_chunks = text_splitter.split_text(transcript)

In [15]:
# Convert text chunks into Document objects

final_documents = [Document(page_content=chunk) for chunk in text_chunks]

In [16]:
# Define the map prompt template

chunks_prompt = """
You are a summarization model using a map-reduce approach. 
Task is to summarize the text provided below. 
You do not change the language of the text. 
Focus only on creating a clear and concise summary while maintaining the original meaning.
<text>
{text}
<text>
Summary:
"""
map_prompt_template = PromptTemplate(input_variables=['text'], template=chunks_prompt)

In [17]:
# Define the final prompt template
final_prompt = """
You are a summarization model using the map-reduce approach. 
Your task is to create a final summary from the text provided below. 

Make sure the summary:
- Is clear and easy to understand.
- Focuses on the main ideas and leaves out unnecessary details.
- Is between 10 to 300 words, depending on the length of the original content.
- Does not change the language of the text.

<text>
{text}
<text>

Final summary:
"""

final_prompt_template = PromptTemplate(input_variables=['text'], template=final_prompt)

In [20]:
# Initialize the LLM

llm = ChatGroq(model="llama-3.3-70b-versatile", groq_api_key=groq_api_key)

In [21]:
# Load the summarization chain

summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=False)

In [22]:
# Show the summarization chain on the final documents

output = summary_chain.run(final_documents)

C:\Users\darsh\AppData\Local\Temp\ipykernel_6804\2179287939.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = summary_chain.run(final_documents)
e:\FX_Internship\Learning_Tasks\Projects\Youtube_Video_Summarization\venv_video_sum\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (4119 > 1024). Running this sequence through the model will result in indexing errors


In [23]:
print(output)

पीटर पांडे के पास एक तोता है जिसका नाम बड़ी है, जो एक लैंग्वेज मॉडल है जो चीजों को कॉपी करने में माहिर है। यह मॉडल कविताएँ लिख सकता है, पोषण संबंधी सलाह दे सकता है, और जटिल कार्यों को पूरा कर सकता है। बड़ी के पास एक सुपर पावर है जिससे वह पीटर के घर के अलावा आसपास के लोगों के घरों में भी जा सकता है और उनके वार्तालाप सुन सकता है, जिससे इसकी क्षमता बढ़ जाती है।

एलएलएम (लार्ज लैंग्वेज मॉडल) एक शक्तिशाली तकनीक है जो चैट जीबीटी और चैट जीपीटी जैसे एप्लिकेशन को शक्ति प्रदान करता है। यह मॉडल बड़ी कंपनियों द्वारा विकसित किए गए हैं और वे विभिन्न कार्य कर सकते हैं। एलएलएम को ट्रेन करने के लिए, विशेषज्ञ उसे सिखाते हैं कि कैसे अच्छी और सिविलाइज्ड भाषा का उपयोग करना है।

एलएलएम एक कंप्यूटर प्रोग्राम है जो चीजों को याद करके और उनका पुनरावृत्ति करके काम करता है, लेकिन यह इंसानों की तरह सब्जेक्टिव अनुभव, भावनाएं और चेतना नहीं रखती है। वीडियो के माध्यम से एलएलएम के बारे में जानकारी प्राप्त की जा सकती है और यदि वीडियो पसंद आया है तो दूसरों के साथ साझा किया जा सकता है।


### Summray Text to Speech using gTTS

In [22]:
#Import All Required Librearies

from gtts import gTTS
import io
from IPython.display import Audio

In [23]:
# Selection of language which is our output language

language = select_lang

In [24]:
myobj = gTTS(text=output, lang=language, slow=False)

In [25]:
# Save the audio in memory

audio_fp = io.BytesIO()
myobj.write_to_fp(audio_fp)
audio_fp.seek(0)


0

In [26]:
#Play the Audio

Audio(audio_fp.read(), autoplay=True)

In [ ]:
# Saving the audio file

# myobj.save("summary_speech.mp3")

In [ ]:
#Play the audio

# Audio("welcome.mp3")